# Loading libraries

In [11]:
import numpy as np 
import pandas as pd

# Importing dataset

In [28]:
df=pd.read_csv("/home/mllab/IRIS.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
X=df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values # X is the feature matrix
Y=df["species"].values   #Y is the target variable

# Splitting the dataset into testing and training set(20-80)

In [13]:
idx=int(0.8*len(X))
X_train=X[:idx]
X_test=X[idx:]
Y_train=Y[:idx]
Y_test=Y[idx:]

# Using Gini_index as splitting criteria

In [14]:
def gini(y):
    count=np.unique(y)
    g=1.0
    for c in count:
        p=(np.sum(y==c))/len(y)
        g=g-p**2
    return g

In [15]:
def best_split(X,Y):
    best=1
    split_threshold=None
    split_feature=None
    for feature in range(len(X[0])):
        threshold=np.unique(X[:,feature])
        for t in threshold:
            left=Y[X[:,feature]<=t]
            right=Y[X[:,feature]>t]
            #if len(left)==0 or len(right)==0:
                #continue
            g=(len(left)/len(Y))*gini(left)+(len(right)/len(Y))*gini(right)
            if(g<best):
                best=g
                split_threshold=t
                split_feature=feature

    return split_feature,split_threshold

In [16]:
class Node:
    def __init__(self,feature=None,threshold=None,left=None,right=None,value=None):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.value=value

In [21]:
def build_tree(X,Y,depth=0,max_dep=5):
    if(len(np.unique(Y))==1 or depth==max_dep):
        return Node(value=max(set(Y),key=list(Y).count))
    feature,threshold=best_split(X,Y)
    if feature is None:
        return Node(value=max(set(Y),key=list(Y).count))
    left=X[:,feature]<=threshold
    right=X[:,feature]>threshold
    left_child=build_tree(X[left],Y[left],depth+1,max_dep=5)
    right_child=build_tree(X[right],Y[right],depth+1,max_dep=5)
    return Node(feature,threshold,left_child,right_child)

In [25]:
def predict_one(x,tree):
    if tree.value is not None:
        return tree.value
    if x[tree.feature]<=tree.threshold:
        return predict_one(x,tree.left)
    else:
        return predict_one(x,tree.right)

In [26]:
def predict(X,tree):
    return np.array([predict_one(x,tree)for x in X])

In [29]:
tree=build_tree(X_train,Y_train,depth=0,max_dep=5)
y_pred=predict(X_test,tree)
acc=np.mean(y_pred==Y_test)
print("Accuracy",acc)

Accuracy 0.7333333333333333
